In [2]:
# !pip install arxiv



In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [7]:
apiwrapper= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)

In [24]:
Wiki = WikipediaQueryRun(api_wrapper=apiwrapper)

In [25]:
Wiki.name

'wikipedia'

In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

In [15]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OllamaEmbeddings(model="gemma"))
retriver = vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023458AD1C70>)

In [18]:
from langchain.tools.retriever import create_retriever_tool
retriver_tools = create_retriever_tool(retriver,"langsmith_search",
                      "search for information about langsmith. For any questions about langsmith use this tool")

In [20]:
retriver_tools.name

'langsmith_search'

In [21]:
# Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [23]:
arxiv_wapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=100)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wapper)
arxiv.name

'arxiv'

In [26]:
tools = [Wiki,arxiv,retriver_tools]

In [27]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\MUSTAFA\\Desktop\\ML\\Projects\\Chatbots\\evenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=100, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='search for information about langsmith. For any questions about langsmith use this tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000023455AA0A40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbedd

In [29]:
#Agents
from langchain.llms import Ollama
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [30]:
llm = Ollama(model="gemma")

In [36]:
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

In [39]:
# Use the agent
response = agent.invoke({"input:tell me about langsmith"})
print(response)



> Entering new AgentExecutor chain...
Action: gemma
Action Input: "tell me about langsmith"
Observation: ## Langsmith

**Who is Langsmith?**

Langsmith is a leading AI-powered language platform that helps businesses and creators generate compelling written content, automate tasks, and improve their overall communication. 

**What does Langsmith do?**

The platform offers a variety of features and functionalities, including:

* **Content creation:** Generate original blog posts, website copy, product descriptions, marketing materials, and more.
* **Translation:** Translate text between multiple languages in real-time.
* **Summarization:** Summarize long pieces of text into concise and informative summaries.
* **ChatGPT integration:** Access the power of ChatGPT's large language model to answer questions, generate creative content, and more.
* **Workflow automation:** Automate tasks like content editing, research, and data entry.

**Benefits of using Langsmith:**

* **Increased product